In [10]:
#nathan
### - Supplier B and Supplier E are the major contributors for the LRR outliers (where LRR>90ppm per Company's goal definition). While Supplier B should be picked up as top priority for root cause investigation and corrective action taken, as it has more Vendor Induced Defects (VID) comparing with Supplier E.¶
### - Chendu factory contributed highest LRR across all factories. It suggests that Chendu factory could be a valuable focus area to investiagte and take action to drive next level LRR performance improvement.

######## Dependency
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import csv
import time



In [11]:
#reading data and preview
data = pd.read_csv("cleaned_data.csv")
city_weather = pd.read_csv("city_weather.csv")
del city_weather["Unnamed: 0"]
city_weather.head(7)

,City,Latitude,Longitude,JAstrt,JAend,FEstrt,FEend,MRstrt,MRend,APstrt,APend,MYstrt,MYend
0,Brasilia,-10.90,-37.55,73.33,72.26,72.88,72.64,73.88,73.24,72.39,75.95,75.20,71.20
1,Chengdu,30.68,104.06,43.09,45.77,52.42,52.12,46.59,66.96,58.79,72.22,75.88,82.95
2,Kunshan,31.39,120.95,39.77,41.05,39.36,50.54,51.55,57.37,65.33,61.03,75.09,77.25
3,Penang,-7.81,113.99,85.69,86.61,84.74,88.19,85.74,88.99,85.80,84.37,85.29,91.96
4,Mumbai,19.01,72.85,84.69,82.02,81.88,82.53,83.94,85.01,84.60,89.12,88.87,89.19
5,Suzhou,33.65,116.96,39.08,35.16,43.88,52.71,53.18,62.07,70.83,73.59,77.73,86.07
6,Taoyuan,24.99,121.30,61.36,77.23,55.90,62.68,70.09,66.47,60.48,86.22,65.72,88.21


In [12]:
data.head()

,MONTH,SUPPLIER,FACTORY,COUNTRY,DPN,LOB,WATT_LEVEL,LR,CONSUMPTION,VID,CND,PID
0,JA,A,Brasilia,Brazil,7FHHV,Consumer,Medium,0,35,0,0,0
1,JA,A,Brasilia,Brazil,7FHHV,Consumer,Medium,0,50,0,0,0
2,JA,A,Brasilia,Brazil,7FHHV,Commercial,Medium,0,158,0,0,0
3,JA,A,Chengdu,China,7FHHV,Consumer,Medium,0,1,0,0,0
4,JA,A,Chengdu,China,7FHHV,Commercial,Medium,0,1,0,0,0


In [13]:
#Adding LRR per country
data_sum = data.groupby("FACTORY").sum()
data_sum["LRR(ppm)"] = data_sum["LR"]/data_sum["CONSUMPTION"]*1000000
data_sum

,LR,CONSUMPTION,VID,CND,PID,LRR(ppm)
FACTORY,,,,,,
Brasilia,64,320881,40,7,17,199.450887
Chengdu,197,3935260,5,96,96,50.060225
Kunshan,0,5399536,0,0,0,0.000000
Mumbai,2,6376,1,0,1,313.676286
Penang,0,152901,0,0,0,0.000000
Suzhou,0,233441,0,0,0,0.000000
Taoyuan,4,94254,4,0,0,42.438517


In [14]:
#Adding average change
#"City","Latitude","Longitude","JAstrt","JAend","JA▲", "FEstrt","FEend","FE▲","MRstrt","MRend","MR▲","APstrt","APend","AP▲","MYstrt","MYend","MY▲"
count = 0
for row in city_weather:
    count += 1
    
    if count == 4:
        last = city_weather[row]
        
    elif count > 4 and count%2 ==1:
            
        col = row +str(count)
        city_weather[col] = city_weather[row] - last
        last = city_weather[row]
        
city_weather
            

,City,Latitude,Longitude,JAstrt,JAend,FEstrt,FEend,MRstrt,MRend,APstrt,APend,MYstrt,MYend,JAend5,FEend7,MRend9,APend11,MYend13
0,Brasilia,-10.90,-37.55,73.33,72.26,72.88,72.64,73.88,73.24,72.39,75.95,75.20,71.20,-1.07,0.38,0.60,2.71,-4.75
1,Chengdu,30.68,104.06,43.09,45.77,52.42,52.12,46.59,66.96,58.79,72.22,75.88,82.95,2.68,6.35,14.84,5.26,10.73
2,Kunshan,31.39,120.95,39.77,41.05,39.36,50.54,51.55,57.37,65.33,61.03,75.09,77.25,1.28,9.49,6.83,3.66,16.22
3,Penang,-7.81,113.99,85.69,86.61,84.74,88.19,85.74,88.99,85.80,84.37,85.29,91.96,0.92,1.58,0.80,-4.62,7.59
4,Mumbai,19.01,72.85,84.69,82.02,81.88,82.53,83.94,85.01,84.60,89.12,88.87,89.19,-2.67,0.51,2.48,4.11,0.07
5,Suzhou,33.65,116.96,39.08,35.16,43.88,52.71,53.18,62.07,70.83,73.59,77.73,86.07,-3.92,17.55,9.36,11.52,12.48
6,Taoyuan,24.99,121.30,61.36,77.23,55.90,62.68,70.09,66.47,60.48,86.22,65.72,88.21,15.87,-14.55,3.79,19.75,1.99


In [15]:
#renaming columns to read easier
city_weather = city_weather.rename(columns = {"JAstrt":"Jan(strt)","JAend":"Jan(end)","JAend5":"Jan▲", "FEstrt":"Feb(strt)",
                                              "FEend":"Feb(end)","FEend7":"Feb▲","MRstrt":"Mar(strt)","MRend":"Mar(end)",
                                              "MRend9":"Mar▲","APstrt":"Apr(strt)","APend":"Apr(end)","APend11":"Apr▲",
                                              "MYstrt":"May(strt)","MYend":"May(end)","MYend13":"May▲"})

#reorganizing columns to make more sense
city_weather = city_weather[["City","Latitude","Longitude","Jan(strt)","Jan(end)","Jan▲","Feb(strt)","Feb(end)","Feb▲",
                             "Mar(strt)","Mar(end)","Mar▲","Apr(strt)","Apr(end)","Apr▲","May(strt)","May(end)","May▲"]]

city_weather


,City,Latitude,Longitude,Jan(strt),Jan(end),Jan▲,Feb(strt),Feb(end),Feb▲,Mar(strt),Mar(end),Mar▲,Apr(strt),Apr(end),Apr▲,May(strt),May(end),May▲
0,Brasilia,-10.90,-37.55,73.33,72.26,-1.07,72.88,72.64,0.38,73.88,73.24,0.60,72.39,75.95,2.71,75.20,71.20,-4.75
1,Chengdu,30.68,104.06,43.09,45.77,2.68,52.42,52.12,6.35,46.59,66.96,14.84,58.79,72.22,5.26,75.88,82.95,10.73
2,Kunshan,31.39,120.95,39.77,41.05,1.28,39.36,50.54,9.49,51.55,57.37,6.83,65.33,61.03,3.66,75.09,77.25,16.22
3,Penang,-7.81,113.99,85.69,86.61,0.92,84.74,88.19,1.58,85.74,88.99,0.80,85.80,84.37,-4.62,85.29,91.96,7.59
4,Mumbai,19.01,72.85,84.69,82.02,-2.67,81.88,82.53,0.51,83.94,85.01,2.48,84.60,89.12,4.11,88.87,89.19,0.07
5,Suzhou,33.65,116.96,39.08,35.16,-3.92,43.88,52.71,17.55,53.18,62.07,9.36,70.83,73.59,11.52,77.73,86.07,12.48
6,Taoyuan,24.99,121.30,61.36,77.23,15.87,55.90,62.68,-14.55,70.09,66.47,3.79,60.48,86.22,19.75,65.72,88.21,1.99


In [17]:
city_weather.to_csv("city_weather_change.csv")


In [37]:
#LRR by Factory
LRR_city = data[["FACTORY","CONSUMPTION","LR","VID","CND","PID"]]
LRR_city = LRR_city.groupby("FACTORY").sum()
LRR_city["LRR (ppm)"] = LRR_grouped["LR"]/LRR_grouped["CONSUMPTION"]*1000000
LRR_city.reset_index()
LRR_city.to_csv("LRR_factory.csv")
LRR_city

,CONSUMPTION,LR,VID,CND,PID,LRR (ppm)
FACTORY,,,,,,
Brasilia,320881,64,40,7,17,199.450887
Chengdu,3935260,197,5,96,96,50.060225
Kunshan,5399536,0,0,0,0,0.000000
Mumbai,6376,2,1,0,1,313.676286
Penang,152901,0,0,0,0,0.000000
Suzhou,233441,0,0,0,0,0.000000
Taoyuan,94254,4,4,0,0,42.438517


In [40]:
#LRR by Supplier
LRR_supplier = data[["SUPPLIER","CONSUMPTION","LR","VID","CND","PID"]]
LRR_supplier = LRR_supplier.groupby("SUPPLIER").sum()
LRR_supplier["LRR (ppm)"] = LRR_supplier["LR"]/LRR_supplier["CONSUMPTION"]*1000000
LRR_supplier


,CONSUMPTION,LR,VID,CND,PID,LRR (ppm)
SUPPLIER,,,,,,
A,3078278,59,1,13,45,19.166560
B,176734,57,40,7,10,322.518587
C,1687936,13,1,1,11,7.701714
D,1968742,72,1,38,33,36.571577
E,3230959,66,7,44,15,20.427372
